# Simultaneous diagonalization

In [2]:
import numpy as np
import numpy.linalg as la

R = [None, None]
V = [None, None]
R[0] = np.array([[0.,          0., - 0.68510884, 0.],
                 [0.,         0.,        0., - 0.68644896],
                 [-0.68510884, 0.,         0.,          0.],
                 [0., - 0.68644896, 0.,          0.]])
# [-0.68644896 -0.68510884  0.68510884  0.68644896]
V[0] = np.array([[0.,        0.70710678, - 0.70710678, 0.],
                 [0.70710678, 0.,  0., - 0.70710678],
                 [0.,      0.70710678, 0.70710678, 0.],
                 [0.70710678, 0.,    0.,     0.70710678]])
R[1] = np.array([[0., - 0.68510884, 0.,        0.],
                 [-0.68510884, 0.,         0.,         0.],
                 [0.,      0.,         0., - 0.68644896],
                 [0.,      0., - 0.68644896, 0.]])
# [-0.68644896 - 0.68510884  0.68510884  0.68644896]
V[1] = np.array([[0.,        0.70710678, - 0.70710678, 0.],
                 [0.,      0.70710678, 0.70710678, 0.],
                 [0.70710678, 0.,  0., - 0.70710678],
                 [0.70710678, 0.,    0.,       0.70710678]])
print(R[0] @ R[1] - R[1] @ R[0])

[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.00183805  0.        ]
 [ 0.         -0.00183805  0.          0.        ]
 [ 0.          0.          0.          0.        ]]


In [5]:
B = V[0].T @ R[1] @ V[0]
la.eigh(B)

(array([-0.68644896, -0.68510884,  0.68510884,  0.68644896]),
 array([[-0.5,  0.5, -0.5,  0.5],
        [-0.5,  0.5,  0.5, -0.5],
        [-0.5, -0.5, -0.5, -0.5],
        [-0.5, -0.5,  0.5,  0.5]]))

### New unitary 'optimization' method

In [1]:
from Hubbard.equalizer import *
import numpy as np
from scipy.optimize import minimize, shgo, dual_annealing

N = 20
R0 = np.array([3, 3, 7.2])

W = HubbardParamEqualizer(N,
                          R0=R0,
                          lattice=np.array([4, 4], dtype=int),
                          lc=(1500, 1500),
                          band=1,
                          dim=3,
                          avg=1 / 2,
                          sparse=True,
                          equalize=False,
                          symmetry=True)

eqV = True
fixed = False
W.eq_label = 'eq'

E, V, parity = eigen_basis(W)
R = locality_mat(W, V[0], parity[0])
np.set_printoptions(precision=4, suppress=True)
A1, U1 = singleband_optimize(W, E[0], V[0], parity[0])
A2, U2 = singleband_diagonalize(W, E[0], V[0], parity[0])

DVR: dx=[0.15 0.15 0.36]w is set.
DVR: n=[20 20 20] is set.
DVR: R0=[3.  3.  7.2]w is set.
['x' 'y' 'z']-reflection symmetry is used.
param_set: trap parameter V0=104.52kHz w=1000nm
Triangular lattice size adjust to: [4 4]
lattice: dx is fixed at: [0.15 0.15 0.36]w
lattice: lattice shape is square
lattice: Full lattice sizes: [4 4]
lattice: lattice constants: [1.5 1.5 1.5]w
DVR: dx=[0.15 0.15 0.36]w is set.
DVR: n=[35 35 20] is set.
DVR: R0=[5.25 5.25 7.2 ]w is set.
H_op: n=[35 35 20] dx=[0.15 0.15 0.36]w p=[1 1 1] Gaussian sparse diagonalization is enabled. Lowest 16 states are to be calculated.
H_op: n=[35 35 20] dx=[0.15 0.15 0.36]w p=[1 1 1] Gaussian operator constructed.
H_solver: diagonalize sparse hermitian matrix.
H_solver: Gaussian Hamiltonian solved. Time spent: 2.49s.
H_solver: eigenstates memory usage: 3.32 MiB.
H_op: n=[35 35 20] dx=[0.15 0.15 0.36]w p=[ 1 -1  1] Gaussian sparse diagonalization is enabled. Lowest 16 states are to be calculated.
H_op: n=[35 35 20] dx=[0.15 

The target function Jacobi angle to minimize is the same as cost function of MLWF Riemannian manifold optimization. The two method is equivalent. The only difference is that in Jacobi angle a closed form is provided for exactly commuting matrices.

However, due to unexact commutativeness of Rx and Ry, given the error from sparse diagonalization, the error grows with the matrix scale, which makes the matrices finally non-commute nuermcally.

In [5]:
%timeit A1, U1 = singleband_optimize(W, E[0], V[0], parity[0])
%timeit A2, U2 = singleband_diagonalize(W, E[0], V[0], parity[0])

Optimizing...
Terminated - min grad norm reached after 307 iterations, 0.08 seconds.

Trap site position of Wannier functions: [ 2  5  4  7  1  9 14 11  2  6 10  3 15 13 12  8]
Order of Wannier function is set to match trap site.
Optimizing...
Terminated - min grad norm reached after 268 iterations, 0.07 seconds.

Trap site position of Wannier functions: [ 8 13  0  1 15  9  2 12  4 14 10 11 15  5  7  6]
Order of Wannier function is set to match trap site.
Optimizing...
Terminated - min grad norm reached after 234 iterations, 0.06 seconds.

Trap site position of Wannier functions: [ 0 14  6 11  3 12  9  1  8  7 13  5  3 15  2  4]
Order of Wannier function is set to match trap site.
Optimizing...
Terminated - min grad norm reached after 315 iterations, 0.08 seconds.

Trap site position of Wannier functions: [14  9 15  7  6  0  1  4  2  5 12 10 11 13 11  3]
Order of Wannier function is set to match trap site.
Optimizing...
Terminated - min grad norm reached after 269 iterations, 0.07 seco

In [7]:
from tools.jacobi_angles import jacobi_angles
import numpy as np

N = 2

# A = np.eye(N)
# B = np.array([[0., 1.], [1., 0.]])
# jacobi_angles(A, B)

# A = np.eye(N)
# C = np.array([[0., -1j], [1j, 0.]])
# jacobi_angles(A, C)

jacobi_angles(C)

Rotation s = 0.7071067811865475
Rotation s = 9.983673087713698e-17
Jacobi angle converged.


(array([[0.7071-0.j    , 0.    -0.7071j],
        [0.    -0.7071j, 0.7071-0.j    ]]),
 array([[-1.],
        [ 1.]]),
 2.684153782866527e-16)

In [6]:
print(np.real(A1))
print(np.real(U1))
print(A2)
print(U2)

[[-36.8455   0.371    0.0358   0.0243   0.371   -0.0785   0.0033  -0.0102
   -0.0358   0.0033  -0.0185   0.0214   0.0243  -0.0102  -0.0214  -0.0102]
 [  0.371  -37.8024   0.2725   0.0061   0.0315  -0.3668  -0.0062  -0.0013
   -0.0111   0.0219  -0.0075   0.007    0.0045  -0.0165  -0.0135  -0.0165]
 [  0.0358   0.2725 -37.8783  -0.1245   0.0111  -0.0142   0.3187  -0.0262
   -0.0046   0.0015  -0.0238   0.0066   0.0022  -0.0059  -0.0094  -0.0059]
 [  0.0243   0.0061  -0.1245 -37.3513   0.0045  -0.008    0.1632   0.6411
   -0.0022   0.0032   0.0069  -0.0234  -0.0307  -0.0179   0.0078  -0.0179]
 [  0.371    0.0315   0.0111   0.0045 -37.8024  -0.3668   0.0219  -0.0165
   -0.2725  -0.0062  -0.0075   0.0135   0.0061  -0.0013  -0.007   -0.0013]
 [ -0.0785  -0.3668  -0.0142  -0.008   -0.3668 -38.9189  -0.2723   0.031
    0.0142  -0.2723   0.0015  -0.0088  -0.008    0.031    0.0088   0.031 ]
 [  0.0033  -0.0062   0.3187   0.1632   0.0219  -0.2723 -38.9655  -0.2399
   -0.0015  -0.0052  -0.2666   0.